# Create new project for analysis and call the project and active map

In [ ]:
#if running this code outside of ArcGIS Pro put the project file path in quotes in place of CURRENT
#if running this project in Pro, open this Notebook and the map that will contain the data for the analysis
aprx = arcpy.mp.ArcGISProject("CURRENT")
mp = aprx.listMaps("Access*")[0]
print(mp.name)

# Add USA Parks to the map
**Last updated: 2024-11-11**

Tasks: 
- Import the ArcGIS API for Python library
- Access ArcGIS Online and the USA Parks item (need to search for the correct item and copy the item ID): https://uw-mad.maps.arcgis.com/home/item.html?id=e49e181ac82c46edac3ae601ebb3ef2d 
- Extract the feature layer and export it to the project file geodatabase - this will save it locally as a File Geodatabase Feature Class and add it to the map

In [ ]:
from arcgis.gis import GIS

In [ ]:
#access ArcGIS Online
portal = GIS("home")

#access USA Parks feature service
parks_id = "e49e181ac82c46edac3ae601ebb3ef2d"

parks_item = portal.content.get(parks_id)
parks_item

In [ ]:
out_parks_fc = "USA_Parks_export"
#put filepath for project geodatabase in the quotes
gdb = r""

#access feature layer
parks_fl = parks_item.layers[0]

#use query to return a feature set object
export = parks_fl.query()

#save feature set as a feature class in the project geodatabase
export.save(gdb, out_parks_fc)

# Add all 2020 TIGER/Line census block shapefiles to the map
**Created by GL, edited by JH, Last updated: 2024-11-11**

2020 Census TIGER/Line Tabulation Block Shapefiles were used for this calculation. Tabulation blocks had population and housing counts for each census block. The record layout can be found here: https://www.census.gov/programs-surveys/geography/technical-documentation/complete-technical-documentation/tiger-geo-line.2020.html#list-tab-GV16DOVS6MWQBLMN86. The shapefiles for each state had to be downloaded and unzipped manually. Download here: https://www.census.gov/geographies/mapping-files/time-series/geo/tiger-line-file.2020.html#list-tab-790442341 

Note: The CT statecode (09) was excluded from the dataframe used to add census block shapefiles to the map.. The census tabulation block GEOIDs for Connecticut still reflect the old CT counties. They need to be crosswalked to the new census block GEOIDs that go with the planning regions (new CT county equivalents). The crosswalk can be found here: https://github.com/CT-Data-Collaborative/2022-block-crosswalk. 

**Tasks:**

- create a dataframe that has following columns:
    - "statecode", two-digit state fipscode for all states except CT (09)
    - "file_path" (for shapefiles)
- loop through "file_path" to add shapefiles to map, i.e., `mp.addDataFromPath(shp_path)`
- crosswalk old CT census block GEOIDs to new CT census block GEOIDs
    - read in csv with crosswalk as a pandas dataframe, drop un-needed fields, convert all columns to strings with leading zeros
    - in pandas dataframe crosswalk, rename block_fips_2022 to GEOID20 (necessary to join to CT 2020 census block file)
    - read in CT 2020 census block shapefile as a spatially enabled dataframe
    - join census block spatially enabled dataframe to crosswalk with a left outer join on GEOID20 (census block spatially enabled dataframe needs to be the left data frame becuase it has more rows and the left outer join will retain those rows; the crosswalk dropped some census blocks that had no population)
    - after join, rename GEOID20 to old_GEOID20 and COUNTYFP20 to old_COUNTYFP20 and then drop GEOID20 and COUNTYFP20
    - then rename block_fips_2022 (new census block fipscodes) to GEOID20 and name a substring of the last three digits of ce_fips_2022 (new county equivalent fipscodes) COUNTYFP20 to match other census block shapefile column names
    - convert the spatially enabled dataframe to a feature class following the naming convention of the other census block shapefiles (Note: it will be a feature class saved in the project geodatabase while the rest of the census block layers are shapefiles saved on external hard drive)

## Create dataframe

In [ ]:
import pandas as pd

In [ ]:
#list of state fips codes
#purposefully excluding state code 09 (CT)
statecode = [str(code).zfill(2) for code in list(range(57)) if code not in (0, 3, 7, 9, 14, 43, 52)]

#create dataframe with a column 'statecode'
df = pd.DataFrame(statecode, columns=["statecode"])

#put the root of the filepath where the TIGER/Line census tabulation blocks are saved before /tl_2020_
dir_base = "/tl_2020_" 

#add columns of 'file_path', 'tabblock', 'intersect'
df["file_path"] = dir_base + df["statecode"] + "_tabblock20.shp" #end of the TIGER/Line census tabulation block shapefile name
#df["tabblock"] = "tl_2020_" + df["statecode"] + "_tabblock20"
#df["intersect"] = "block_" + df["statecode"] + "_intersect"

#print(df)

df.shape

In [ ]:
print(df.head())

## Add shapefiles to map 

In [ ]:
#try the first two states
for i in range(2):
    print(df.iloc[i,1])
    mp.addDataFromPath(df.iloc[i,1])

In [ ]:
#helpful to pause drawing on map before running 
#loop through all states
for i in range(len(df)):
    print(df.iloc[i,1])
    mp.addDataFromPath(df.iloc[i,1])

## Crosswalk CT census block GEOIDS

In [ ]:
#read in crosswalk csv - put filepath of wherever crosswalk .csv is saved in quotes
ct_crosswalk = pd.read_csv(r"")

#show contents of csv
ct_crosswalk

In [ ]:
 #subsetting dataframe to just the columns needed 
ct_crosswalk_2 = ct_crosswalk[['block_fips_2020', 'block_fips_2022', 'ce_fips_2022']]

ct_crosswalk_2.head()

In [ ]:
#convert all columns to strings, add leading zeros, change column name block_fips_2020 to GEOID20 so it can be joined to the census block geography file
ct_crosswalk_2 = ct_crosswalk_2.astype(str)

ct_crosswalk_2['GEOID20'] = ct_crosswalk_2['block_fips_2020'].str.zfill(15)
ct_crosswalk_2['block_fips_2022'] = ct_crosswalk_2['block_fips_2022'].str.zfill(15)
ct_crosswalk_2['ce_fips_2022'] = ct_crosswalk_2['ce_fips_2022'].str.zfill(5)

ct_crosswalk_2.head()

In [ ]:
#retain only the block_fips_2022, GEOID20, and COUNTYFP20 column names
ct_crosswalk_3 = ct_crosswalk_2[['GEOID20', 'block_fips_2022', 'ce_fips_2022']]

ct_crosswalk_3.head()

In [ ]:
#convert tl_2020_09_tabblock_20 (using filepath from whevever the census tabulation block shapefiles are saved on your local machine) to spatially enabled dataframe to manipulate columns 
CT = pd.DataFrame.spatial.from_featureclass(r'\tl_2020_09_tabblock20.shp')

CT.head()

In [ ]:
#join crosswalk (ct_crosswalk_3) to census block geography file (CT)
#left outer join using the merge function, CT should be left dataframe
CT_join = pd.merge(
    CT, ct_crosswalk_3, how = "left", on = ["GEOID20"]
    )

#show contents of join
CT_join

In [ ]:
#rename GEOID20 (old census block fipscode) and COUNTYFP20 (old county fipscode)
CT_join['old_COUNTYFP20'] = CT_join['COUNTYFP20']
CT_join['old_GEOID20'] = CT_join['GEOID20']

CT_join.head()

In [ ]:
#drop GEOID20 (old census block fipscode) and COUNTYFP20 (old county fipscode)
CT_join_2 = CT_join.drop(columns = ['GEOID20', 'COUNTYFP20'])

CT_join_2.head()

In [ ]:
#rename block_fips_2022 (new census block fipscodes) to GEOID20 
#take substring of last three characters of ce_fips_2022 (new county equivalent fipscodes) and name COUNTYFP20 
#to match other census block shapefile columns
CT_join_2['GEOID20'] = CT_join_2['block_fips_2022']
CT_join_2['COUNTYFP20'] = CT_join_2['ce_fips_2022'].str.slice(2, 5)

CT_join_3 = CT_join_2.drop(columns = ['block_fips_2022', 'ce_fips_2022'])

#CT_join_2.head()
CT_join_3.head()

In [ ]:
#convert spatial data frame back to a feature class in project geodatabase
CT_join_3.copy().spatial.to_featureclass(location = gdb + r"\tl_2020_09_tabblock20")

2024-11-11 Note: I did a visual inspection of the tl_2020_09_tabblock20 feature class attribute table, and there are 65 census blocks missing from the crosswalk. The github page for the crosswalk noted that census blocks in 2020 census tracts that contained only water were excluded. Confirmed in the attribute table that all blocks that didn't have a join with a block in the crosswalk have 0 population (sort geoid20 field ascending to see this). 

# Change coordinate system
**Last updated: 2024-11-11**

The USA Parks feature class from the Esri Living Atlas is in the WGS 1984 geographic coordinate system (GCS). 
The TIGER/Line Census Block and County boundary files are in the NAD 1983 GCS. 
For this calculation, USA Parks will be re-projected to the NAD 1983 GCS to match the Census files. 

Leave all inputs unprojected - there is no map projection that is appropriate for the entire U.S.

In [ ]:
#Parameters were set with the tool interface and the Python command was copied to ensure information about the geographic transformation was correct
#arcpy.management.Project(in_dataset, out_dataset, out_coor_system, {transform_method}, {in_coor_system}, {preserve_shape}, {max_deviation}, {vertical})

arcpy.management.Project(
    in_dataset="USA_Parks_export",
    out_dataset="USA_Parks_NAD1983",
    out_coor_system='GEOGCS["GCS_North_American_1983",DATUM["D_North_American_1983",SPHEROID["GRS_1980",6378137.0,298.257222101]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]]',
    transform_method="WGS_1984_(ITRF00)_To_NAD_1983",
    in_coor_system='GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]]',
    preserve_shape="NO_PRESERVE_SHAPE",
    max_deviation=None,
    vertical="NO_VERTICAL"
)

# Create 1/2 mile buffer around parks 
Use the geodesic method, which accounts for the shape of the Earth and is more appropriate for inputs that cover large regions (see tool documentation https://pro.arcgis.com/en/pro-app/2.8/tool-reference/analysis/pairwise-buffer.htm and "How Buffer Analysis Works" https://pro.arcgis.com/en/pro-app/latest/tool-reference/analysis/how-buffer-analysis-works.htm). With the geodesic method, distances are calculated between two points on a curved surface (the geoid). 

In [ ]:
arcpy.analysis.PairwiseBuffer("USA_Parks_NAD1983", "Parks_GeodesicBuffer", "0.5 Miles", "ALL", None, "GEODESIC")

# Intersect Parks_GeodesicBuffer with 2020 TIGER/Line census blocks
**Created by GL, edited by JH, Last updated: 2024-11-12**

**Tasks:**

- create a dataframe that has following columns:
    - "statecode", two-digit state fipscode for all states (**Note**: you must run the step to create a new dataframe with statecode defined to include CT before running the intersect step)
    - "tabblock" and "intersect"
- loop through "tabblock" and "intersect" for arcpy.analysis.Intersect 

## Create dataframe

In [ ]:
import pandas as pd

In [ ]:
#code for dataframe copied from above, this time need all states in statecode
#list of state fips codes
statecode_2 = [str(code).zfill(2) for code in list(range(57)) if code not in (0, 3, 7, 14, 43, 52)]

#create dataframe with a column 'statecode'
df_2 = pd.DataFrame(statecode_2, columns=["statecode"])

#add columns of 'tabblock', 'intersect'
#df["file_path"] = dir_base + df["statecode"] + "_tabblock20.shp" #end of the TIGER/Line census tabulation block shapefile name
df_2["tabblock"] = "tl_2020_" + df_2["statecode"] + "_tabblock20"
df_2["intersect"] = "block_" + df_2["statecode"] + "_intersect"

df_2.shape

In [ ]:
print(df_2)

## Loop: arcpy.analysis.Intersect

In [ ]:
#try the first 3 
#arcpy.analysis.Intersect(in_features, out_feature_class, {join_attributes}, {cluster_tolerance}, {output_type})

for i in (0, 1, 2):
    print(f"{df_2.iloc[i,1]}, {df_2.iloc[i,2]}")
    #arcpy.analysis.Intersect([df_2.iloc[i,1],"Parks_GeodesicBuffer"], df_2.iloc[i,2], "ALL")

In [ ]:
#loop through all the states
#arcpy.analysis.Intersect(in_features, out_feature_class, {join_attributes}, {cluster_tolerance}, {output_type})

for i in range(len(df_2)):
    print(f"{df_2.iloc[i,1]}, {df_2.iloc[i,2]}")
    arcpy.analysis.Intersect([df_2.iloc[i,1],"Parks_GeodesicBuffer"], df_2.iloc[i,2], "ALL")

# Aggregate total pop and pop with access to parks to county level
Find the total population in each county by taking the sum of POP20 field for all census blocks in a county (denominator). Find the population that has access to a park in each county by taking the sum of the POP20 field for all census blocks that intersect with Parks_GeodesicBuffer (numerator). Divide population with access to parks by total population. 

**Created by JH, edited from code written by GL, Last updated 2024-11-22**

**Tasks:**

- create a dataframe that has following columns:
    - "statecode", two-digit state fipscode for all states
    - "tabblock" and "intersect"
    - "totalPop" and "accessPop"
- loop through aggregating the population with access to parks and the total population to the county level

## Create dataframe

In [ ]:
import pandas as pd
#if you haven't already

In [ ]:
#list of state fips codes including CT (09)
statecode = [str(code).zfill(2) for code in list(range(57)) if code not in (0, 3, 7, 14, 43, 52)]

#create dataframe with a column 'statecode' 
df_3 = pd.DataFrame(statecode, columns=["statecode"])

#add columns of 'tabblock', 'intersect', 'totalPop', 'accessPop'
df_3["tabblock"] = "tl_2020_" + df_3["statecode"] + "_tabblock20"
df_3["intersect"] = "block_" + df_3["statecode"] + "_intersect"
df_3["totalPop"] = "county_" + df_3["statecode"] + "_totalPop"
df_3["accessPop"] = "county_" + df_3["statecode"] + "_accessPop"

print(df_3)

#df_3.shape

In [ ]:
print(df_3.head())

## Loop: arcpy.analysis.Statistics and arcpy.analysis.AlterField

In [ ]:
#aggregate total county population - input is all census blocks - and rename "SUM_POP20" to "parks_denominator"
#arcpy.analysis.Statistics(in_table, out_table, {statistics_fields}, {case_field}, {concatenation_separator})
#arcpy.management.AlterField(in_table, field, {new_field_name}, {new_field_alias}, {field_type}, {field_length}, {field_is_nullable}, {clear_field_alias})

for i in range(len(df_3)):
    print(f"{df_3.iloc[i,1]}, {df_3.iloc[i,3]}")
    arcpy.analysis.Statistics((df_3.iloc[i,1]), (df_3.iloc[i,3]), [["POP20", "SUM"]], ["STATEFP20", "COUNTYFP20"])
    print(f"{df_3.iloc[i,3]}")
    arcpy.management.AlterField((df_3.iloc[i,3]), "SUM_POP20", "v179_denominator", "v179_denominator")

In [ ]:
#aggregate county population with access to parks - input is intersected census blocks - and rename "SUM_POP20" to "parks_numerator"

for i in range(len(df_3)):
    print(f"{df_3.iloc[i,2]}, {df_3.iloc[i,4]}")
    arcpy.analysis.Statistics((df_3.iloc[i,2]), (df_3.iloc[i,4]), [["POP20", "SUM"]], ["STATEFP20", "COUNTYFP20"])
    print(f"{df_3.iloc[i,4]}")
    arcpy.management.AlterField((df_3.iloc[i,4]), "SUM_POP20", "v179_numerator", "v179_numerator")

# Join tables or fields and calculate measure

Join the parks_numerator field for each state to the county_[stateFIPS]_totalPop table for each respective state and then calculate the measure (numerator divided by denominator). Joining the numerator field to the table that contains the denominator ensures that no counties are dropped. If no census blocks in a county intersected with a park, that county is not included in the accessPop table and will have a missing value for the measure.

Since this is being done state by state, county fipscode (COUNTYFP) can be used as the join field 

- Join Field tool: https://pro.arcgis.com/en/pro-app/latest/tool-reference/data-management/join-field.htm 
- Add Field tool: https://pro.arcgis.com/en/pro-app/latest/tool-reference/data-management/add-field.htm
- Calculate Field tool: https://pro.arcgis.com/en/pro-app/latest/tool-reference/data-management/calculate-field.htm

**Created by JH, edited from code written by GL, Last updated 2024-11-22**

**Tasks:**

- loop through joining the numerator field to the county_[stateFIPS]_totalPop table
- loop through adding a new field where calculated measure will go
- merge all county_[stateFIPS]_totalPop tables together into final measure table
- calculate the measure

## Loop: arcpy.management.JoinField

In [ ]:
print(df_3.head())

In [ ]:
#join numerator field to table with total population - input table is table with total pop field, input join field is county fipscode,
#join table is table with pop that has access to parks, join table field is county fipscode, transfer field is parks_numerator
#arcpy.management.JoinField(in_data, in_field, join_table, join_field, {fields}, {fm_option}, {field_mapping})

for i in range(len(df_3)):
    print(f"{df_3.iloc[i,3]}, {df_3.iloc[i,4]}")
    arcpy.management.JoinField((df_3.iloc[i,3]), "COUNTYFP20", (df_3.iloc[i,4]), "COUNTYFP20", ["v179_numerator"])

## Loop: arcpy.management.AddField

In [ ]:
#add new field
#arcpy.management.AddField(in_table, field_name, field_type, {field_precision}, {field_scale}, {field_length}, {field_alias}, {field_is_nullable}, {field_is_required}, {field_domain})

for i in range(len(df_3)):
    print(f"{df_3.iloc[i,3]}")
    arcpy.management.AddField((df_3.iloc[i,3]), "v179_rawvalue", "DOUBLE", None, None, None, "v179_rawvalue", "NULLABLE")

## Merge all the tables together before doing the final measure calculation

In [ ]:
#merge all county_[stateFIPS]_totalPop tables together 
#arcpy.management.Merge(inputs, output, {field_mappings}, {add_source})

arcpy.management.Merge(["county_01_totalPop", "county_02_totalPop", "county_04_totalPop", "county_05_totalPop", "county_06_totalPop", "county_08_totalPop",
                       "county_09_totalPop", "county_10_totalPop", "county_11_totalPop", "county_12_totalPop", "county_13_totalPop", "county_15_totalPop",
                       "county_16_totalPop", "county_17_totalPop", "county_18_totalPop", "county_19_totalPop", "county_20_totalPop", "county_21_totalPop",
                       "county_22_totalPop", "county_23_totalPop", "county_24_totalPop", "county_25_totalPop", "county_26_totalPop", "county_27_totalPop",
                       "county_28_totalPop", "county_29_totalPop", "county_30_totalPop", "county_31_totalPop", "county_32_totalPop", "county_33_totalPop",
                       "county_34_totalPop", "county_35_totalPop", "county_36_totalPop", "county_37_totalPop", "county_38_totalPop", "county_39_totalPop",
                       "county_40_totalPop", "county_41_totalPop", "county_42_totalPop", "county_44_totalPop", "county_45_totalPop", "county_46_totalPop",
                       "county_47_totalPop", "county_48_totalPop", "county_49_totalPop", "county_50_totalPop", "county_51_totalPop", "county_53_totalPop",
                       "county_54_totalPop", "county_55_totalPop", "county_56_totalPop"], "parks_access")

**After the merge, there are 3,145 counties. New counties in AK are included (02063 and 02066) and a null county in CT (a result of the census block crosswalk not containing the census blocks that were in tracts that have no population). There is no COUNTYFP20 value for this row and the numerator and denominator are 0 - just needs to be deleted.**

In [ ]:
#select CT row that has no COUNTYFP value and a numerator and denominator of 0
#Parameters were set with the tool interface and the Python command was copied

arcpy.management.SelectLayerByAttribute(
    in_layer_or_view="parks_access",
    selection_type="NEW_SELECTION",
    where_clause="STATEFP20 = '09' And v179_denominator = 0 And v179_numerator = 0",
    invert_where_clause=None
)

In [ ]:
#delete selected CT row that has no COUNTYFP value and a numerator and denominator of 0
#arcpy.management.DeleteRows(in_rows)

arcpy.management.DeleteRows(in_rows="parks_access")

## Calculate the measure

In [ ]:
#calculate measure - input is merged table that contains all counties 
#arcpy.management.CalculateField(in_table, field, expression, {expression_type}, {code_block}, {field_type}, {enforce_domains})

arcpy.management.CalculateField("parks_access", "v179_rawvalue", "!v179_numerator! / !v179_denominator!", 
                                "PYTHON3", None, "DOUBLE")

Counties are assigned a missing value when no park locations have been identified in the ESRI parks datasets. In contrast, counties are assigned a 0% when they have a park location but the county population does not live within the defined buffers of that park."

**The null values reflect counties where there are no parks (based on our data from Esri USA Parks) - i.e. no census blocks in the county intersect with the buffered areas around a park** 

# Final processing 
**Last updated by JH 2024-11-25**

**Tasks:**
- Create a fipscode field, delete the FREQUENCY field, and rename the STATEFP20 and COUNTYFP20 fields
- Add in old Connecticut counties and flag field for Connecticut counties
- Calculate state and national values
- Merge the county, state, and national value datasets together to create the final dataset
- Save final dataset to P drive

## Editing fields with ArcGIS Pro tools

In [ ]:
#create fipscode field 
arcpy.management.AddField("parks_access", "fipscode", "TEXT", None, None, 5, "fipscode", "NULLABLE")

#concatenate STATEFP20 and COUNTYFP20
arcpy.management.CalculateField("parks_access", "fipscode", "!STATEFP20! + !COUNTYFP20!", 
                                "PYTHON3", None, "TEXT")

In [ ]:
#delete FREQUENCY
arcpy.management.DeleteField("parks_access", ["FREQUENCY"])

In [ ]:
#rename STATEFP20 
arcpy.management.AlterField("parks_access", "STATEFP20", "statecode", "statecode")

#rename COUNTYFP20 
arcpy.management.AlterField("parks_access", "COUNTYFP20", "countycode", "countycode")


Took the parks_access table at this point and joined it to the 2022 Census 1:500,000 county cartographic boundary national file to create a national map to share with the research team. **Update**: Code was re-run to create parks_access on 2024-11-22, but all datapoints should be the same. An earlier version of the dataset was used to create the initial national map. 

Dataset still needs state and national values, rows for the old Connecticut counties will null values, and the flag_CT field indicating which CT counties have data available. 

## Add in old CT counties and flag field for CT
**Last updated by JH 2024-11-22**

Using pandas library. 

- Convert parks_access table to a pandas dataframe by saving as a csv and them importing a csv (easiest way to do this based on what I've searched online)
- Add flag_CT column
- Create dataframe for old CT counties
- Merge old CT counties with parks_access measure dataset


In [ ]:
#arcpy.conversion.TableToTable(in_rows, out_path, out_name, {where_clause}, {field_mapping}, {config_keyword})

out_path = r"" #filtpath in quotes of wherever you want to store your outputs

arcpy.conversion.TableToTable("parks_access", out_path, "parks_access.csv")

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
#read in csv file with measure data

data_path = out_path + r"\parks_access.csv"

parks_df = pd.read_csv(data_path, dtype={"fipscode":str, "statecode":str, "countycode":str})

parks_df

In [ ]:
#drop OID column

parks_df = parks_df.drop(columns = ["OID_"])

parks_df

In [ ]:
#add empty flag_CT column 

parks_df["flag_CT"] = " "

parks_df.head()

In [ ]:
#assign "A" in flag_CT for new CT counties (planning regions)

parks_df['flag_CT'].loc[parks_df['statecode'] == '09'] = 'A'

parks_df[parks_df['statecode'] == '09']

In [ ]:
#create df for old CT counties

oldCT_df = pd.DataFrame({'countycode': ['001', '003', '005', '007', '009', '011', '013', '015']})
oldCT_df['statecode'] = '09'
oldCT_df['fipscode'] = oldCT_df['statecode'] + oldCT_df['countycode']
oldCT_df['v179_denominator'] = np.nan
oldCT_df['v179_numerator'] = np.nan
oldCT_df['v179_rawvalue'] = np.nan
oldCT_df['flag_CT'] = 'U'

oldCT_df

In [ ]:
#merge old CT counties with parks_access, Access to parks is measure number v179

v179_county = pd.concat([parks_df, oldCT_df], ignore_index = True)

In [ ]:
v179_county.dtypes

In [ ]:
#reset index

v179_county = v179_county.reset_index(drop = True)

v179_county

In [ ]:
#sort based on fipscode (sometimes need to run the #reset index and #sort based on fipscodes blocks of code twice)

v179_county = v179_county.sort_values('fipscode')

v179_county[v179_county['statecode'] == '09']

## Calculating state and national values 

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
#calculate national value

nat_num = v179_county['v179_numerator'].sum()
nat_den =  v179_county['v179_denominator'].sum()

v179_nat = pd.DataFrame({
    'statecode': '00',
    'countycode': '000',
    'fipscode': '00000',
    'v179_denominator': [nat_den],
    'v179_numerator': [nat_num],
    'v179_rawvalue': [nat_num / nat_den],
    'flag_CT': ' '
})

v179_nat

In [ ]:
#calculate state values 

state_num = v179_county.groupby('statecode')['v179_numerator'].sum().reset_index(name = 'v179_numerator')
state_den = v179_county.groupby('statecode')['v179_denominator'].sum().reset_index(name = 'v179_denominator')

#join state numerator and denominator dataframes 
v179_state = pd.merge(
    state_num, state_den, how = 'left', on = ['statecode']
    )

#add other columns 
v179_state['countycode'] = '000'
v179_state['fipscode'] = v179_state['statecode'] + v179_state['countycode']
v179_state['v179_rawvalue'] = v179_state['v179_numerator'] / v179_state['v179_denominator']
v179_state['flag_CT'] = ' '

#show contents of join
v179_state

The .reset_index() with a column name specified is necessary to turn the results of the groupby() into a dataframe. 

You can check if an object is a dataframe using type()

## Create final v179 dataset and save to the P drive

In [ ]:
#checking datatypes in each dataframe to be merged together 

print("County:",
     v179_county.dtypes)

print("State:",
      type(v179_state),
      v179_state.dtypes)

print("National:",
     v179_nat.dtypes)

In [ ]:
#combine dataframes

v179_df = pd.concat([v179_county, v179_state, v179_nat], ignore_index = True)

v179_df

In [ ]:
v179_df = v179_df.sort_values('fipscode')
v179_df = v179_df.reset_index(drop = True)

v179_df

In [ ]:
#export pandas v179_df as a csv to my files 

v179_df.to_csv(out_path + r'\v179.csv', index = False)

**Note**: I tried to bring the .csv file into Pro to then use the Table to SAS tool to save the final dataset as SAS dataset, but Pro was not maintaining leading zeros for the statecode, countycode, and fipscode fields. SAS will be used to save the final dataset as a SAS dataset on the P drive. 